# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS leonskeyspace
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy' , 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('leonskeyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
field_idx = {field_name : idx for idx, field_name in enumerate(['artist', 'first_name', 'gender',\
                                                     'item_in_session', 'last_name','song_length',\
                                                     'level','location','session_id','song_title',\
                                                     'user_id'])}

## Query 1
---
- Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- Query 1 uses sessionId and itemInSession in WHERE clause.
    - Therefore, session_id and item_in_session comprises primary key.

In [9]:
create_table = """
CREATE TABLE IF NOT EXISTS song_info_session 
(
    session_id int,
    item_in_session int,
    artist text,
    song_title text,
    song_length decimal,
    PRIMARY KEY (session_id, item_in_session)
)
"""

try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[field_idx['session_id']]),\
                                int(line[field_idx['item_in_session']]),\
                                line[field_idx['artist']],\
                                line[field_idx['song_title']],\
                                float(line[field_idx['song_length']])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
query = """
SELECT artist, song_title, song_length FROM song_info_session
WHERE session_id = 338 AND item_in_session = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query2
---
- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- Query 2 uses user_id and session_id in WHERE clause and item_in_session for sorting.
    - Therefore, user_id, session_id and item_in_session comprises primary key.

In [13]:
create_table = """
CREATE TABLE IF NOT EXISTS song_playlist_session 
(
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    first_name text,
    last_name text,
    song_title text,
    PRIMARY KEY((user_id, session_id), item_in_session)
)
WITH CLUSTERING ORDER BY (item_in_session DESC);
"""

try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (user_id, session_id, item_in_session, artist, first_name, last_name, song_title)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[field_idx['user_id']]),\
                                int(line[field_idx['session_id']]),\
                                int(line[field_idx['item_in_session']]),\
                                line[field_idx['artist']],\
                                line[field_idx['first_name']],\
                                line[field_idx['last_name']],\
                                line[field_idx['song_title']]))

In [15]:
#I selected item_in_session to see if query result was sorted by item_in_session as required. 
query = """
SELECT artist, song_title, item_in_session, first_name, last_name FROM song_playlist_session
WHERE user_id = 10 AND session_id = 182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist,",",row.song_title,",",row.item_in_session,",", row.first_name,",", row.last_name)

Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , 3 , Sylvie , Cruz
Sebastien Tellier , Kilometer , 2 , Sylvie , Cruz
Three Drives , Greece 2000 , 1 , Sylvie , Cruz
Down To The Bone , Keep On Keepin' On , 0 , Sylvie , Cruz


## Query 3
---
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- Query 3 uses song_title in WHERE clause.
    - For uniqueness of each row, primary key also includes user id.

In [16]:
create_table = """
CREATE TABLE IF NOT EXISTS song_playlist_user 
(
    song_title text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY (song_title, user_id)
)
"""

try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_user (song_title, user_id,first_name, last_name)"
        query = query + " VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[field_idx['song_title']],\
                                int(line[field_idx['user_id']]),\
                                line[field_idx['first_name']],\
                                line[field_idx['last_name']]))

In [18]:
query = """
SELECT first_name, last_name FROM song_playlist_user
WHERE song_title = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name,",",row.last_name)

Jacqueline , Lynch
Tegan , Levine
Sara , Johnson


### Drop the tables before closing out the sessions

In [19]:
try:
    session.execute("DROP TABLE song_info_session")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE song_playlist_session")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE song_playlist_user")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()